# Export Predictions to CSV (for R)

In [38]:
import os
import matplotlib.pyplot as plt
import yaml
#from ct_classifier/ct_classifier.train import create_dataloader, load_model       # NOTE: since we're using these functions across files, it could make sense to put them in e.g. a "util.py" script.
import torch
import pandas as pd
import numpy as np
from torch import softmax as softmax

In [39]:
#os.getcwd()
os.chdir('/home/magali/ct_classifier/ct_classifier')

In [30]:
from train import load_model
from train import create_dataloader

## Parameters

In [48]:
config = '/home/magali/ct_classifier/configs/exp_resnet18_sexRoeDeer.yaml'
split = 'val'
BEST_MODEL_PATH = '/home/magali/ct_classifier/best_model_80.pt' #model needs to have a number before the pt



## Load Data

In [49]:
# load config
print(f'Using config "{config}"')
cfg = yaml.safe_load(open(config, 'r'))


# setup entities
dl_val = create_dataloader(cfg, split='val')

# load model
#model = torch.load(BEST_MODEL_PATH)
#model = load_model(cfg)[0] #takes first model, refer to bets model here
#model = load_model(cfg)[BEST_MODEL_PATH]
#BEST_MODEL_PATH = '/home/magali/ct_classifier/best_models/sex_basic_run1_epoch54.pt'
#model_instance, _ = load_model(cfg, model_path=BEST_MODEL_PATH)




Using config "/home/magali/ct_classifier/configs/exp_resnet18_sexRoeDeer.yaml"
val number of images 502 labels 502 images covered 502


In [51]:
model_instance, start_epoch = load_model(cfg, model_path=BEST_MODEL_PATH)


Loading model from /home/magali/ct_classifier/best_model_80.pt


In [ ]:
#model_instance, start_epoch = load_model(cfg, model_path=BEST_MODEL_PATH)

#state = torch.load(BEST_MODEL_PATH, map_location='cpu')
#model_instance.load_state_dict(state)



## Visualize

This is up to you to figure out now. :)

In [52]:
print(type(model_instance))
#print(model)
#print(prediction)

<class 'model.CustomResNet18'>


In [53]:
#device = cfg['device']
#model.to(device)

model_instance.eval()

CustomResNet18(
  (feature_extractor): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affin

In [54]:
all_pred_labels = []
all_labels = []
all_img_path = []

def tuple_to_list(obj):
    if isinstance(obj, tuple):
        return [tuple_to_list(item) for item in obj]
    return obj

with torch.no_grad():               # don't calculate intermediate gradient steps: we don't need them, so this saves memory and is faster
        for idx, (data, labels, img_path) in enumerate(dl_val):

            # put data and labels on device
            #data, labels = data.to(device), labels.to(device)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


            print(data.shape)
            #print(model.classifier.device)
            print(data.device)
            print(device)
            # forward pass
            prediction = model_instance(data)
            #print(prediction[0])
            #print(img_path[0])

            # If you want to track labels and paths as well, you can continue to do that
            pred_label = prediction
            all_pred_labels = all_pred_labels + pred_label.cpu().tolist()
            print(all_pred_labels)
            all_labels = all_labels + labels.tolist()
            print(all_labels)

            # Convert tuple to numpy array
            #img_path_arr = np.array(img_path)
            #img_path_lst = tuple_to_list(img_path)

            #all_img_path = all_img_path + img_path_arr.tolist()

            # Flatten the img_path tuple
            img_path_lst = tuple_to_list(img_path)
            all_img_path.extend(img_path_lst)

            # Print sizes for debugging
            print(f"Iteration {idx}: all_pred_labels size: {len(all_pred_labels)}, all_labels size: {len(all_labels)}, all_img_path size: {len(all_img_path)}")

            
        # After the loop, print or return the summed predictions
        print(all_pred_labels, all_labels, all_img_path)




torch.Size([128, 3, 224, 224])
cpu
cuda
[[-0.7849259972572327, -0.4843542277812958], [0.5678979158401489, 0.49243468046188354], [0.602979302406311, -0.1941165030002594], [-0.5287777185440063, 0.5763750672340393], [0.6192408204078674, 0.23720374703407288], [0.820496141910553, -0.14750662446022034], [-0.3752259612083435, -0.19809958338737488], [0.03845406696200371, 1.0326241254806519], [-0.28687211871147156, -0.27989763021469116], [-0.34275609254837036, -0.09574345499277115], [0.01385733112692833, -0.057747043669223785], [-0.11093246936798096, 0.2004472017288208], [1.040534496307373, -0.819264829158783], [0.6004704833030701, -0.6571635007858276], [0.4182543158531189, -0.4555027186870575], [0.04451471194624901, 0.3397483229637146], [-1.849339246749878, -0.022125251591205597], [0.4795863926410675, -0.7749927043914795], [0.7878220081329346, -0.3113569915294647], [-0.4639720618724823, -0.7083667516708374], [0.1689298152923584, -0.9123594164848328], [-0.7617489099502563, 0.5651080012321472], 

In [55]:
print(all_labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 

Create dataframes with predictions, labels and image id

In [56]:
label_df= pd.DataFrame(all_labels, columns=['Labels'])
pred_df = pd.DataFrame(all_pred_labels, columns=['pred_female', 'pred_male'])
img_id_df = pd.DataFrame(all_img_path, columns=['Image path'])


In [57]:
print(pred_df)

     pred_female  pred_male
0      -0.784926  -0.484354
1       0.567898   0.492435
2       0.602979  -0.194117
3      -0.528778   0.576375
4       0.619241   0.237204
..           ...        ...
497     0.136475  -0.126528
498     0.084412  -0.026706
499    -1.314963   0.509645
500    -0.302876   0.175051
501    -0.061268  -0.455234

[502 rows x 2 columns]


In [58]:
print(img_id_df)

                                            Image path
0    /home/magali/CV4Ecology-summer-school/Prototyp...
1    /home/magali/CV4Ecology-summer-school/Prototyp...
2    /home/magali/CV4Ecology-summer-school/Prototyp...
3    /home/magali/CV4Ecology-summer-school/Prototyp...
4    /home/magali/CV4Ecology-summer-school/Prototyp...
..                                                 ...
497  /home/magali/CV4Ecology-summer-school/Prototyp...
498  /home/magali/CV4Ecology-summer-school/Prototyp...
499  /home/magali/CV4Ecology-summer-school/Prototyp...
500  /home/magali/CV4Ecology-summer-school/Prototyp...
501  /home/magali/CV4Ecology-summer-school/Prototyp...

[502 rows x 1 columns]


In [59]:
result = pd.concat([label_df, pred_df, img_id_df], axis=1, ignore_index=False)
print(result)

     Labels  pred_female  pred_male  \
0         0    -0.784926  -0.484354   
1         0     0.567898   0.492435   
2         0     0.602979  -0.194117   
3         0    -0.528778   0.576375   
4         0     0.619241   0.237204   
..      ...          ...        ...   
497       0     0.136475  -0.126528   
498       0     0.084412  -0.026706   
499       0    -1.314963   0.509645   
500       0    -0.302876   0.175051   
501       0    -0.061268  -0.455234   

                                            Image path  
0    /home/magali/CV4Ecology-summer-school/Prototyp...  
1    /home/magali/CV4Ecology-summer-school/Prototyp...  
2    /home/magali/CV4Ecology-summer-school/Prototyp...  
3    /home/magali/CV4Ecology-summer-school/Prototyp...  
4    /home/magali/CV4Ecology-summer-school/Prototyp...  
..                                                 ...  
497  /home/magali/CV4Ecology-summer-school/Prototyp...  
498  /home/magali/CV4Ecology-summer-school/Prototyp...  
499  /home/magali/

convert predictions to with softmax

In [60]:
# Define a function to apply torch.softmax to a list of scores
#def apply_softmax(scores):
#    tensor_scores = torch.tensor(scores)
#    softmax_scores = torch.softmax(tensor_scores, dim=0).tolist()  # Using dim=0 for 1D tensor
#    return softmax_scores

# Apply the function to the 'prediction' column
#result['pred_female_softmax'] = result['pred_female'].apply(torch.softmax)
#result['pred_male_softmax'] = result['pred_male'].apply(apply_softmax)

# Convert DataFrame columns to a tensor
tensor_predictions = torch.tensor(result[['pred_female', 'pred_male']].values)

# Compute softmax along the last dimension (across columns for each row)
softmax_tensor = torch.softmax(tensor_predictions, dim=1)

# Convert the tensor back to a DataFrame
df_softmax = pd.DataFrame(softmax_tensor.numpy(), columns=['pred_female_softmax', 'pred_male_softmax'])

# Concatenate the two DataFrames along axis=1
result = pd.concat([result, df_softmax], axis=1)

print(result)


     Labels  pred_female  pred_male  \
0         0    -0.784926  -0.484354   
1         0     0.567898   0.492435   
2         0     0.602979  -0.194117   
3         0    -0.528778   0.576375   
4         0     0.619241   0.237204   
..      ...          ...        ...   
497       0     0.136475  -0.126528   
498       0     0.084412  -0.026706   
499       0    -1.314963   0.509645   
500       0    -0.302876   0.175051   
501       0    -0.061268  -0.455234   

                                            Image path  pred_female_softmax  \
0    /home/magali/CV4Ecology-summer-school/Prototyp...             0.425418   
1    /home/magali/CV4Ecology-summer-school/Prototyp...             0.518857   
2    /home/magali/CV4Ecology-summer-school/Prototyp...             0.689353   
3    /home/magali/CV4Ecology-summer-school/Prototyp...             0.248776   
4    /home/magali/CV4Ecology-summer-school/Prototyp...             0.594364   
..                                                 ...   

In [61]:
print(result)

     Labels  pred_female  pred_male  \
0         0    -0.784926  -0.484354   
1         0     0.567898   0.492435   
2         0     0.602979  -0.194117   
3         0    -0.528778   0.576375   
4         0     0.619241   0.237204   
..      ...          ...        ...   
497       0     0.136475  -0.126528   
498       0     0.084412  -0.026706   
499       0    -1.314963   0.509645   
500       0    -0.302876   0.175051   
501       0    -0.061268  -0.455234   

                                            Image path  pred_female_softmax  \
0    /home/magali/CV4Ecology-summer-school/Prototyp...             0.425418   
1    /home/magali/CV4Ecology-summer-school/Prototyp...             0.518857   
2    /home/magali/CV4Ecology-summer-school/Prototyp...             0.689353   
3    /home/magali/CV4Ecology-summer-school/Prototyp...             0.248776   
4    /home/magali/CV4Ecology-summer-school/Prototyp...             0.594364   
..                                                 ...   

In [62]:
result.to_csv('../predictions/predictions_sex_RoeDeer.csv', index=True)
